In [1]:
import os 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 

In [2]:
# radar plots - one color for video data, one color for video data 
# is general "gestalt" of video the same as that for the mat data?  

# why stride time, cadence, double and single support ? 
    # stride time and cadence most accurate vid metrics 
    # double and single support from mat most strongly correlated with EDSS and T25FW 

In [3]:
# version 
version = '006'

In [4]:
# PWS Zeno videos 
zv_pws_all_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                               version, 
                               '000_merged_cleaned_data', 
                                'zv_bw_merged_gait_vertical_PWS_1_raw.csv') 
zv_pws_all_df = pd.read_csv(zv_pws_all_path, index_col = 0)

In [5]:
# PWS Zeno videos 
zv_fw_all_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                           version, 
                              '000_merged_cleaned_data',
                           'zv_bw_merged_gait_vertical_FW_1_raw.csv') 
zv_fw_all_df = pd.read_csv(zv_fw_all_path, index_col = 0)

In [6]:
# home videos 
hv_all_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                           version, 
                           '000_merged_cleaned_data',
                           'hv_bw_merged_raw.csv') 
hv_all_df = pd.read_csv(hv_all_path, index_col = 0)

In [7]:
# output path 
output_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                           version, 
                           'radar_plots') 

if not os.path.exists(output_path): 
    os.makedirs(output_path)

if not os.path.exists(os.path.join(output_path, 'home_videos')): 
    os.makedirs(os.path.join(output_path, 'home_videos'))

if not os.path.exists(os.path.join(output_path, 'zeno_videos', 'pws')): 
    os.makedirs(os.path.join(output_path, 'zeno_videos', 'pws'))

if not os.path.exists(os.path.join(output_path, 'zeno_videos', 'fw')): 
    os.makedirs(os.path.join(output_path, 'zeno_videos', 'fw'))

In [8]:
def radar_plot_mat_vs_vid(categories, values_1, values_2, values_1_label, values_2_label): 
    # Number of variables/categories
    num_vars = len(categories)

    # Compute angle for each category
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]  # Complete the loop

    # Add first value to close the radar plot
    values_1 += values_1[:1]
    values_2 += values_2[:1]

    # Create the plot
    fig, ax = plt.subplots(figsize=(5, 5), subplot_kw=dict(polar=True))

    # Plot the first dataset
    ax.plot(angles, values_1, color='royalblue', linewidth=3, linestyle='solid', label=values_1_label, alpha = 0.75)
    ax.fill(angles, values_1, color='royalblue', alpha=0.25)

    # Plot the second dataset
    ax.plot(angles, values_2, color='orange', linewidth=3, linestyle='solid', label=values_2_label, alpha = 0.75)
    ax.fill(angles, values_2, color='orange', alpha=0.25)

    # Add category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize = '15')
    # Make x-axis tick labels bold
    for tick in ax.get_xticklabels():
        tick.set_fontweight('bold')

    # For normalized plots - Set the radial axis range and labels
 #   ax.set_ylim(0, 125)
  #  ax.set_yticks([.25, .5, .75, 1.00, 1.25, 1.5, 1.75, 2, 2.25])
   # ax.set_yticklabels(['', '', '', '1.00', '', '', '', '2.00', ''], 
                    #   color='gray', size=10)

    # Add legend
    ax.legend(loc='right', fontsize = '15', prop={'weight': 'bold'}, bbox_to_anchor=[1.55, 0.7])

    # Show the plot
    plt.close()
    return fig 

In [9]:
# Zeno FW videos vs FW in person zeno mat 
categories_fw = ['Stride Time',
                  'Cadence',
                  'Stride Width',
                  'Double Support',
                  'Single Support' 
                  ]

for video_i, current_vid in enumerate(zv_fw_all_df['video_id_date_name_pose_zv']): 
    # mat data 
    values_1_fw = [zv_fw_all_df.loc[video_i, 'FW_stridetimesecmean'],
                zv_fw_all_df.loc[video_i, 'FW_cadencestepsminmean'], 
                zv_fw_all_df.loc[video_i, 'FW_stridewidthcmmean'],  
                zv_fw_all_df.loc[video_i, 'FW_totaldsupportmean'], 
                zv_fw_all_df.loc[video_i, 'FW_singlesupportmean']
               ]

    # video data (home videos) 
    values_2_fw = [zv_fw_all_df.loc[video_i, 'stride_time_mean_sec_pose_zv'],  
                zv_fw_all_df.loc[video_i, 'mean_cadence_step_per_min_pose_zv'], 
                zv_fw_all_df.loc[video_i, 'stride_width_mean_cm_pose_zv'], 
                zv_fw_all_df.loc[video_i, 'tot_dsupport_per_mean_pose_zv'], 
                zv_fw_all_df.loc[video_i, 'singlesupport_per_mean_pose_zv']
                ]
                                      
    # radar plot 
    current_fig = radar_plot_mat_vs_vid(categories = categories_fw, 
                                        values_1 = values_1_fw, 
                                        values_2 = values_2_fw, 
                                        values_1_label = 'FW Zeno Walkway', 
                                        values_2_label = 'FW Video')
    
    current_fig.savefig(os.path.join(output_path, 'zeno_videos', 'fw',
                                     f'zv_fw_{current_vid}_radar_plot.png'), 
                        bbox_inches='tight', dpi=300)

In [10]:
# Zeno PWS videos vs PWS in person zeno mat 
categories_pws = ['Stride Time',
                  'Cadence',
                  'Stride Width',
                  'Double Support',
                  'Single Support' 
                  ]

for video_i, current_vid in enumerate(zv_pws_all_df['video_id_date_name_pose_zv']): 
    # mat data 
    values_1_pws = [zv_pws_all_df.loc[video_i, 'PWS_stridetimesecmean'],
                zv_pws_all_df.loc[video_i, 'PWS_cadencestepsminmean'], 
                zv_pws_all_df.loc[video_i, 'PWS_stridewidthcmmean'], 
                zv_pws_all_df.loc[video_i, 'PWS_totaldsupportmean'], 
                zv_pws_all_df.loc[video_i, 'PWS_singlesupportmean']
               ]

    # video data (home videos) 
    values_2_pws = [zv_pws_all_df.loc[video_i, 'stride_time_mean_sec_pose_zv'],  
                zv_pws_all_df.loc[video_i, 'mean_cadence_step_per_min_pose_zv'],  
                zv_pws_all_df.loc[video_i, 'stride_width_mean_cm_pose_zv'], 
                zv_pws_all_df.loc[video_i, 'tot_dsupport_per_mean_pose_zv'], 
                zv_pws_all_df.loc[video_i, 'singlesupport_per_mean_pose_zv']
                ]
                                      
    # radar plot 
    current_fig = radar_plot_mat_vs_vid(categories = categories_pws, 
                                        values_1 = values_1_pws, 
                                        values_2 = values_2_pws, 
                                        values_1_label = 'PWS Zeno Walkway', 
                                        values_2_label = 'PWS Videos')
    
    current_fig.savefig(os.path.join(output_path, 'zeno_videos', 'pws',
                                     f'zv_pws_{current_vid}_radar_plot.png'), 
                        bbox_inches='tight', dpi=300)

In [12]:
# home videos vs PWS in person zeno mat
categories_hv = ['Stride Time',
                  'Cadence',
                  'Stride Width',
                  'Double Support',
                  'Single Support']


for video_i, current_vid in enumerate(hv_all_df['video_id_date_name_pose_hv']): 
    # mat data 
    values_1_hv = [hv_all_df.loc[video_i, 'PWS_stridetimesecmean'],
                hv_all_df.loc[video_i, 'PWS_cadencestepsminmean'],  
                hv_all_df.loc[video_i, 'PWS_stridewidthcmmean'], 
                hv_all_df.loc[video_i, 'PWS_totaldsupportmean'], 
                hv_all_df.loc[video_i, 'PWS_singlesupportmean']
               ]

    # video data (home videos) 
    values_2_hv = [hv_all_df.loc[video_i, 'stride_time_mean_sec_pose_hv'],  
                hv_all_df.loc[video_i, 'mean_cadence_step_per_min_pose_hv'],  
                hv_all_df.loc[video_i, 'stride_width_mean_cm_pose_hv'], 
                hv_all_df.loc[video_i, 'tot_dsupport_per_mean_pose_hv'], 
                hv_all_df.loc[video_i, 'singlesupport_per_mean_pose_hv']
                ]
                                      
    # radar plot 
    current_fig = radar_plot_mat_vs_vid(categories = categories_hv, 
                                        values_1 = values_1_hv, 
                                        values_2 = values_2_hv, 
                                        values_1_label = 'Zeno Walkway PWS', 
                                        values_2_label = 'Home Videos')
    
    current_fig.savefig(os.path.join(output_path, 'home_videos', f'hv_{current_vid}_radar_plot.png'), 
                        bbox_inches='tight', dpi=300)
    

KeyError: 7